# Merge data sets into training and validation sets without droping signals

In [1]:
import pandas as pd
import numpy as np

In [2]:
# load the training data sets
landsat_df = pd.read_csv('Benchmark/landsat_features_training.csv')
terraclimate_df = pd.read_csv('Benchmark/terraclimate_features_training.csv')
wq_df = pd.read_csv('Benchmark/water_quality_training_dataset.csv')
landsat_df.shape, terraclimate_df.shape, wq_df.shape

((9319, 9), (9319, 4), (9319, 6))

In [3]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_datasets(dataset1,dataset2,dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2,dataset3], axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

In [4]:
df = combine_datasets(landsat_df, terraclimate_df, wq_df)
df.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,pet,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0


In [5]:
df.shape, df.columns, df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9319 entries, 0 to 9318
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Latitude                       9319 non-null   float64
 1   Longitude                      9319 non-null   float64
 2   Sample Date                    9319 non-null   object 
 3   nir                            8234 non-null   float64
 4   green                          8234 non-null   float64
 5   swir16                         8234 non-null   float64
 6   swir22                         8234 non-null   float64
 7   NDMI                           8234 non-null   float64
 8   MNDWI                          8234 non-null   float64
 9   pet                            9319 non-null   float64
 10  Total Alkalinity               9319 non-null   float64
 11  Electrical Conductance         9319 non-null   float64
 12  Dissolved Reactive Phosphorus  9319 non-null   f

((9319, 13),
 Index(['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16',
        'swir22', 'NDMI', 'MNDWI', 'pet', 'Total Alkalinity',
        'Electrical Conductance', 'Dissolved Reactive Phosphorus'],
       dtype='object'),
 None,
           Latitude    Longitude           nir         green        swir16  \
 count  9319.000000  9319.000000   8234.000000   8234.000000   8234.000000   
 mean    -28.474988    26.868414  14045.485426   9983.213141  13567.459315   
 std       2.760282     3.535164   2953.223626   2778.780177   3348.517657   
 min     -34.405833    17.730278   3992.000000   4045.000000   3672.500000   
 25%     -30.160091    26.126667  12723.625000   9370.000000  11760.625000   
 50%     -28.058889    27.409060  14183.000000   9801.000000  13704.250000   
 75%     -26.861111    29.245556  15513.875000  10286.000000  15425.625000   
 max     -22.225556    32.325000  65535.000000  65535.000000  65535.000000   
 
              swir22         NDMI        MNDWI  

In [6]:
df.isna().sum()

Latitude                            0
Longitude                           0
Sample Date                         0
nir                              1085
green                            1085
swir16                           1085
swir22                           1085
NDMI                             1085
MNDWI                            1085
pet                                 0
Total Alkalinity                    0
Electrical Conductance              0
Dissolved Reactive Phosphorus       0
dtype: int64

In [7]:
# Merging the validation data sets
# load the validation data sets
landsat_val_df = pd.read_csv('Benchmark/landsat_features_validation.csv')
terraclimate_val_df = pd.read_csv('Benchmark/terraclimate_features_validation.csv')
landsat_val_df.head(), terraclimate_val_df.head()

(    Latitude  Longitude Sample Date      nir    green   swir16   swir22  \
 0 -32.043333  27.822778  01-09-2014  15229.0  12868.0  14797.0  12421.0   
 1 -33.329167  26.077500  16-09-2015      NaN      NaN      NaN      NaN   
 2 -32.991639  27.640028  07-05-2015  16221.0   9304.5  12536.5   9958.0   
 3 -34.096389  24.439167  07-02-2012      NaN      NaN      NaN      NaN   
 4 -32.000556  28.581667  01-10-2014   9125.0  11100.5   9455.0   8711.0   
 
        NDMI     MNDWI  
 0  0.014388 -0.069727  
 1       NaN       NaN  
 2  0.128123 -0.147979  
 3       NaN       NaN  
 4 -0.017761  0.080052  ,
     Latitude  Longitude Sample Date        pet
 0 -32.043333  27.822778  01-09-2014  161.90001
 1 -33.329167  26.077500  16-09-2015  177.60000
 2 -32.991639  27.640028  07-05-2015  158.40001
 3 -34.096389  24.439167  07-02-2012  130.00000
 4 -32.000556  28.581667  01-10-2014  152.50000)

In [8]:
landsat_val_df.shape, terraclimate_val_df.shape

((200, 9), (200, 4))

In [9]:
validation_df = pd.concat([landsat_val_df, terraclimate_val_df], axis=1)
validation_df = validation_df.loc[:, ~validation_df.columns.duplicated()]
validation_df.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,pet
0,-32.043333,27.822778,01-09-2014,15229.0,12868.0,14797.0,12421.0,0.014388,-0.069727,161.90001
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN,NaN,NaN,NaN,177.60000
2,-32.991639,27.640028,07-05-2015,16221.0,9304.5,12536.5,9958.0,0.128123,-0.147979,158.40001
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,NaN,NaN,NaN,130.00000
4,-32.000556,28.581667,01-10-2014,9125.0,11100.5,9455.0,8711.0,-0.017761,0.080052,152.50000


In [10]:
df.to_csv('Training_Dataset.csv', index = False)
validation_df.to_csv('Validation_Dataset.csv', index = False)

In [11]:
df.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,pet,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0


In [12]:
df.describe()

,Latitude,Longitude,nir,green,swir16,swir22,NDMI,MNDWI,pet,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
count,9319.000000,9319.000000,8234.000000,8234.000000,8234.000000,8234.000000,8234.000000,8234.000000,9319.000000,9319.000000,9319.000000,9319.000000
mean,-28.474988,26.868414,14045.485426,9983.213141,13567.459315,11425.538377,0.021374,-0.144268,175.166082,119.108208,485.004146,43.525338
std,2.760282,3.535164,2953.223626,2778.780177,3348.517657,2548.193535,0.077897,0.097646,29.469867,74.692591,341.937736,50.980194
min,-34.405833,17.730278,3992.000000,4045.000000,3672.500000,3634.000000,-0.328293,-0.300487,52.700000,4.800000,15.120000,5.000000
25%,-30.160091,26.126667,12723.625000,9370.000000,11760.625000,9839.500000,-0.036869,-0.211270,156.100000,55.811000,207.050000,10.000000
50%,-28.058889,27.409060,14183.000000,9801.000000,13704.250000,11265.250000,0.021549,-0.167901,172.500000,113.300000,402.000000,20.000000
75%,-26.861111,29.245556,15513.875000,10286.000000,15425.625000,12895.500000,0.073297,-0.104677,193.100000,170.230000,693.000000,48.000000
max,-22.225556,32.325000,65535.000000,65535.000000,65535.000000,31202.500000,0.567905,0.590974,270.800020,361.676000,1506.000000,195.000000


In [15]:
df.drop(columns=['Sample Date']).corr()

,Latitude,Longitude,nir,green,swir16,swir22,NDMI,MNDWI,pet,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
Latitude,1.000000,0.624468,-0.042791,0.010520,0.121183,0.137749,-0.233496,-0.116846,-0.107426,0.318495,0.115727,0.257010
Longitude,0.624468,1.000000,0.050125,-0.012702,0.076827,0.060012,-0.057101,-0.123490,-0.604577,0.036686,-0.213808,0.022743
nir,-0.042791,0.050125,1.000000,0.562378,0.722686,0.631083,0.141624,-0.432936,-0.158514,0.015579,0.037289,-0.004762
green,0.010520,-0.012702,0.562378,1.000000,0.593462,0.553481,-0.129511,0.128332,0.044490,0.061622,0.028293,0.008321
swir16,0.121183,0.076827,0.722686,0.593462,1.000000,0.953234,-0.554963,-0.677372,-0.018378,0.191913,0.144659,0.059861
swir22,0.137749,0.060012,0.631083,0.553481,0.953234,1.000000,-0.633755,-0.624925,0.050784,0.211446,0.159797,0.065900
NDMI,-0.233496,-0.057101,0.141624,-0.129511,-0.554963,-0.633755,1.000000,0.504447,-0.171257,-0.274070,-0.177506,-0.085012
MNDWI,-0.116846,-0.123490,-0.432936,0.128332,-0.677372,-0.624925,0.504447,1.000000,0.119165,-0.154614,-0.141569,-0.048319
pet,-0.107426,-0.604577,-0.158514,0.044490,-0.018378,0.050784,-0.171257,0.119165,1.000000,0.263515,0.322811,0.062566
Total Alkalinity,0.318495,0.036686,0.015579,0.061622,0.191913,0.211446,-0.274070,-0.154614,0.263515,1.000000,0.692436,0.316777


In [16]:
df.drop(columns=['Sample Date']).corr()

,Latitude,Longitude,nir,green,swir16,swir22,NDMI,MNDWI,pet,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
Latitude,1.000000,0.624468,-0.042791,0.010520,0.121183,0.137749,-0.233496,-0.116846,-0.107426,0.318495,0.115727,0.257010
Longitude,0.624468,1.000000,0.050125,-0.012702,0.076827,0.060012,-0.057101,-0.123490,-0.604577,0.036686,-0.213808,0.022743
nir,-0.042791,0.050125,1.000000,0.562378,0.722686,0.631083,0.141624,-0.432936,-0.158514,0.015579,0.037289,-0.004762
green,0.010520,-0.012702,0.562378,1.000000,0.593462,0.553481,-0.129511,0.128332,0.044490,0.061622,0.028293,0.008321
swir16,0.121183,0.076827,0.722686,0.593462,1.000000,0.953234,-0.554963,-0.677372,-0.018378,0.191913,0.144659,0.059861
swir22,0.137749,0.060012,0.631083,0.553481,0.953234,1.000000,-0.633755,-0.624925,0.050784,0.211446,0.159797,0.065900
NDMI,-0.233496,-0.057101,0.141624,-0.129511,-0.554963,-0.633755,1.000000,0.504447,-0.171257,-0.274070,-0.177506,-0.085012
MNDWI,-0.116846,-0.123490,-0.432936,0.128332,-0.677372,-0.624925,0.504447,1.000000,0.119165,-0.154614,-0.141569,-0.048319
pet,-0.107426,-0.604577,-0.158514,0.044490,-0.018378,0.050784,-0.171257,0.119165,1.000000,0.263515,0.322811,0.062566
Total Alkalinity,0.318495,0.036686,0.015579,0.061622,0.191913,0.211446,-0.274070,-0.154614,0.263515,1.000000,0.692436,0.316777
